In [1]:
order=1

In [2]:
# import libraries
import glob                               #can be used to find the target files
import matplotlib.pyplot as plt
import nibabel as nib                   # to read and write neuron imaging data 
import numpy as np                        # for matrix calculating
import subprocess
from PIL import Image 
import os,shutil

from scipy import signal
from skimage import color, img_as_ubyte, io

In [3]:
file_script = 'results_script1'
if not os.path.exists(file_script):
    os.mkdir(file_script)

In [4]:
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/BVECTOR.scheme','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_FA.nii.gz','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_V3.nii.gz','results_script1')

CMD = 'shutil.copy(“fsltensor_V1.nii.gz”,”results_script1”)'
subprocess.call(CMD,shell=True)


In [5]:
cd results_script1

/home/zhiwei/Desktop/semester-projects/results_script1


## generate original tractography and get fib_original, hdr_original

In [6]:
data = nib.load('fsltensor_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask.nii')

In [7]:
CMD='tckgen fsltensor_V3.nii.gz origin0.015.tck -algorithm FACT -seed_image mask.nii -mask mask.nii -select 3000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [8]:
CMD = 'TractConverter.py -i origin0.015.tck -o origin0.015.trk -a fsltensor_FA.nii.gz'
subprocess.call(CMD,shell=True)

0

In [9]:
fib_original,hdr_original=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/origin0.015.trk')

## generate synthetic CLARITY data

In [10]:
CMD='tckmap origin0.015.tck syn_CLARITY.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0


## calculate structure tensor

In [11]:
def doggen(sigma=None, X=None, Y=None, Z=None):
    halfsize = np.ceil(3 * np.max(sigma))       #3*the maximum number of sequence sigma, ceil: shangxian
    x = np.arange(-halfsize,halfsize+1)         # sequence from -halfsize to halfsize+1

    if len(sigma) == 1:
        if X is None:
            X = x
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) )
    if len(sigma) == 2:
        if X is None or Y is None:
            [X,Y] =np.meshgrid(x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) )
    if len(sigma) == 3:
        if X is None or Y is None or Z is None:
            [X,Y,Z] =np.meshgrid(x,x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) ) * np.exp( (-1)*np.power(Z,2)/(2 * np.power(sigma[2],2)))
    if len(sigma) > 3:
        print ('Only support up to dimension 3')
    
    return k /np.sum(np.abs(k))

def gradCompute(img, dogsigma):
    dogkercc = doggen([dogsigma, dogsigma, dogsigma])
    dogkerrr = np.transpose( dogkercc, (1, 0, 2) )
    dogkerzz = np.transpose( dogkercc, (0, 2, 1) )

    gcc = signal.convolve(img, dogkercc, mode = 'valid')
    grr = signal.convolve(img, dogkerrr, mode = 'valid')
    gzz = signal.convolve(img, dogkerzz, mode = 'valid')

    # Gradient products
    gp = type('', (), {})()
    gp.gprrrr = grr * grr
    gp.gprrcc = grr * gcc
    gp.gprrzz = grr * gzz
    gp.gpcccc = gcc * gcc
    gp.gpcczz = gcc * gzz
    gp.gpzzzz = gzz * gzz

    # Gradient amplitude
    ga = np.sqrt(gp.gprrrr + gp.gpcccc + gp.gpzzzz)

    return ga, gp



## the optimal patch_size to retrieve the exact size of CLARITY as the originally synthetic one!! do not change this anymore!!

In [12]:
patch_size = 6.6

In [13]:

img_stack = nib.load('syn_CLARITY.nii.gz')
imgstack = img_stack.get_fdata()/255.0


# select sigma
dogsigma = [0.6]
vox_size = [15, 15, 15] #um
#define the voxel dimension to apply tensor
voxel_dim = np.array([patch_size,patch_size,patch_size]) # x,y,z in voxel dimension
volume_dim = (np.array(imgstack.shape)/voxel_dim).astype(np.int) # in voxel dimesion: removed voxel at the border

fsl_tensor = np.zeros((volume_dim[0], volume_dim[1], volume_dim[2], 6))
for x in range (0,volume_dim[0]):
    for y in range (0,volume_dim[1]):
        for z in range (0, volume_dim[2]):
            [ga,gp] = gradCompute(imgstack[int(x*voxel_dim[0]):int((x+1)*voxel_dim[0]), int(y*voxel_dim[1]):int((y+1)*voxel_dim[1]), int(z*voxel_dim[2]):int((z+1)*voxel_dim[2])], dogsigma)

            lowthreh = np.percentile(ga.reshape(-1), 20)
            highthreh = np.percentile(ga.reshape(-1), 80)

            mask = np.ones(ga.shape)
            mask[ga  < lowthreh] = 0
            mask[ga  > highthreh] = 0

            gprrrr = np.sum(gp.gprrrr[mask==1])
            gprrcc = np.sum(gp.gprrcc[mask==1])
            gprrzz = np.sum(gp.gprrzz[mask==1])
            gpcccc = np.sum(gp.gpcccc[mask==1])
            gpcczz = np.sum(gp.gpcczz[mask==1])
            gpzzzz = np.sum(gp.gpzzzz[mask==1])

            fsl_tensor[x,y,z] = np.array([[gprrrr, gprrcc, gprrzz,gpcccc, gpcczz, gpzzzz]])
affine = np.eye(4)
affine[0,0]=vox_size[0]*voxel_dim[0]*0.001
affine[1,1]=vox_size[1]*voxel_dim[1]*0.001
affine[2,2]=vox_size[2]*voxel_dim[2]*0.001
img_fsl_tensor = nib.Nifti1Image(fsl_tensor, affine)
nib.save(img_fsl_tensor, 'fsltensor_ftrial.nii.gz')
CMD='fsl5.0-fslmaths fsltensor_ftrial.nii.gz -tensor_decomp fsltensor_ftrial'
subprocess.call(CMD, shell=True)

/home/zhiwei/.local/lib/python2.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


0

## generate the mask for subsequent tractography generation

In [14]:
data = nib.load('fsltensor_ftrial_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask_syn.nii')

##  generate the tractography and subsequent CLARITY based on the synthetic CLARITY images

In [15]:
number = 100000

In [16]:
CMD='tckgen fsltensor_ftrial_V3.nii.gz before0.015.tck -algorithm FACT -seed_image mask_syn.nii -mask mask_syn.nii -select 100000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [17]:
CMD = 'TractConverter.py -i before0.015.tck -o before0.015.trk -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [18]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/before0.015.trk')

In [19]:
for items in range(len(fib_original)):
    fib_before.append(fib_original[items])
nib.trackvis.write('fibers_before_all.trk',fib_before,hdr_before)

In [20]:
CMD = 'TractConverter.py -i fibers_before_all.trk -o fibers_before_all.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [21]:
CMD='tckmap fibers_before_all.tck syn_CLARITY_before.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## function for comparison of two tractography on a voxel base

In [22]:
def compClarity(volume1,volume2):
    square_error = 0.0
    for x in range(volume1.shape[0]):
        for y in range(volume1.shape[1]):
            for z in range(volume1.shape[2]):
                square_error = square_error + np.square(volume1[x,y,z]-volume2[x,y,z])
    return square_error

## apply COMMIT to the generated tractography

In [23]:
path_commit = '/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT'
path = '/home/zhiwei/Desktop/semester-projects/results_script1'

In [24]:
trk_file_before = '/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before_all.trk'

In [25]:
from commit import trk2dictionary
trk2dictionary.run(filename_trk   = trk_file_before ,path_out= path_commit )
import commit
commit.core.setup()


-> Creating the dictionary from tractogram:
	* Segment position = COMPUTE INTERSECTIONS
	* Fiber shift X    = 0.000 (voxel-size units)
	* Fiber shift Y    = 0.000 (voxel-size units)
	* Fiber shift Z    = 0.000 (voxel-size units)
	* Points to skip   = 0
	* Min segment len  = 1.00e-03
	* Do not blur fibers
	* Loading data:
		* tractogram
			- 28 x 26 x 5
			- 0.0990 x 0.0990 x 0.0990
			- 103000 fibers
		* no mask specified to filter IC compartments
		* no dataset specified for EC compartments
		* output written to "/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT"
	* Generate tractogram matching the dictionary: 
	  [ 102746 fibers kept ]
   [ 19.1 seconds ]


In [26]:
mit=commit.Evaluation('.','results_script1')

In [27]:
mit.load_data('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY.nii.gz','/home/zhiwei/Desktop/semester-projects/results_script1/BVECTOR.scheme')
mit.set_model('VolumeFractions')
hasISO=False
mit.model.set(hasISO)
mit.generate_kernels(regenerate=True)


-> Loading data:
	* DWI signal...
		- dim    = 191 x 178 x 39 x 1
		- pixdim = 0.015 x 0.015 x 0.015
	* Acquisition scheme...
		- 1 samples, 1 shells
		- 0 @ b=0 , 1 @ b=1000.0
   [ 0.0 seconds ]

-> Preprocessing:
	* There are no b0 volume(s) for normalization... [ min=0.00,  mean=0.25, max=22.00 ]
   [ 0.0 seconds ]

-> Simulating with "Volume fractions" model:
   [ 0.0 seconds ]


In [28]:
mit.load_kernels()
mit.load_dictionary(path_commit)
mit.set_threads(1)


-> Resampling LUT for subject "results_script1":
	* Keeping all b0 volume(s)... [ OK ]
	* Normalizing... [ OK ]
   [ 0.2 seconds ]

-> Loading the dictionary:
	* segments from the tracts... [ 102746 fibers and 5464173 segments ]
	* segments from the peaks...  [ 0 segments ]
	* isotropic contributions...  [ 3582 voxels ]
	* post-processing...          [ OK ]
   [ 2.3 seconds ]

-> Distributing workload to different threads:
	* number of threads : 1
	* A operator...  [ OK ]
	* A' operator... [ OK ]
   [ 0.0 seconds ]


In [29]:
mit.build_operator()


-> Building linear operator A:
   [ 5.1 seconds ]


In [30]:
mit.fit(max_iter = 100000000)
## mit.fit(tol_fun = 1e-7, max_iter = 100000000)


-> Fit model

      |     ||Ax-y||     |  Cost function    Abs error      Rel error    |     Abs x          Rel x
------|------------------|-----------------------------------------------|------------------------------
   1  |   3.4057632e+03  |  5.7996116e+06  4.2942021e+07  7.4042925e+00  |  1.9823562e+02  1.1248671e+00
   2  |   2.2902986e+03  |  2.6227338e+06  3.1768778e+06  1.2112849e+00  |  4.9833596e+01  3.6499296e-01
   3  |   1.5499421e+03  |  1.2011603e+06  1.4215735e+06  1.1835002e+00  |  3.7426819e+01  3.5016715e-01
   4  |   1.0626775e+03  |  5.6464178e+05  6.3651854e+05  1.1272962e+00  |  2.8019690e+01  3.2921080e-01
   5  |   7.4293468e+02  |  2.7597597e+05  2.8866581e+05  1.0459817e+00  |  2.0952910e+01  3.0264295e-01
   6  |   5.3347823e+02  |  1.4229951e+05  1.3367646e+05  9.3940207e-01  |  1.5722831e+01  2.7300409e-01
   7  |   3.9526895e+02  |  7.8118769e+04  6.4180743e+04  8.2157904e-01  |  1.1888037e+01  2.4296485e-01
   8  |   3.0257238e+02  |  4.5775023e+04  3.

  77  |   1.3173062e+01  |  8.6764777e+01  2.8239185e-01  3.2546830e-03  |  7.9533410e-02  1.9589347e-02
  78  |   1.3152677e+01  |  8.6496460e+01  2.6831692e-01  3.1020566e-03  |  7.7793814e-02  1.9187020e-02
  79  |   1.3133362e+01  |  8.6242597e+01  2.5386247e-01  2.9435856e-03  |  7.6047458e-02  1.8776335e-02
  80  |   1.3114921e+01  |  8.6000581e+01  2.4201637e-01  2.8141248e-03  |  7.4821075e-02  1.8488113e-02
  81  |   1.3097186e+01  |  8.5768137e+01  2.3244392e-01  2.7101431e-03  |  7.3533464e-02  1.8179459e-02
  82  |   1.3080088e+01  |  8.5544355e+01  2.2378165e-01  2.6159721e-03  |  7.2358763e-02  1.7893904e-02
  83  |   1.3063528e+01  |  8.5327882e+01  2.1647316e-01  2.5369569e-03  |  7.1242842e-02  1.7618508e-02
  84  |   1.3047582e+01  |  8.5119702e+01  2.0817969e-01  2.4457286e-03  |  7.0047996e-02  1.7319627e-02
  85  |   1.3032471e+01  |  8.4922647e+01  1.9705587e-01  2.3204160e-03  |  6.8215901e-02  1.6859721e-02
  86  |   1.3018150e+01  |  8.4736117e+01  1.8652984e-0

In [31]:
mit.save_results(save_coeff=True)


-> Saving results to "Results_VolumeFractions/*":
	* configuration and results:
		- pickle...  [ OK ]
		- txt...  [ OK ]
	* fitting errors:
		- RMSE...  [ 0.072 +/- 0.201 ]
		- NRMSE... [ 0.019 +/- 0.118 ]
	* voxelwise contributions:
		- intra-axonal [ OK ]
		- extra-axonal [ OK ]
		- isotropic    [ OK ]
   [ 1.1 seconds ]


In [32]:
trk_file = os.path.join(path_commit,'dictionary_TRK_fibers.trk')
fib_tmp,trk_hdr_tmp=nib.trackvis.read(trk_file)

## generate tractography after applying COMMIT with weight > 0, and get fib_after,hdr_after

In [33]:
f = open('/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT/Results_VolumeFractions/xic.txt')


In [34]:
data = f.read().splitlines()
data = map(eval,data)
j=0
for items in range(len(fib_tmp)-1,-1,-1):
    if data[j] == 0:
        del fib_tmp[items]
    j=j+1
nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [35]:
fib_after,hdr_after=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk')

In [36]:
CMD = 'TractConverter.py -i fibers_after.trk -o fibers_after.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [37]:
CMD='tckmap fibers_after.tck syn_CLARITY_after.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## downsample fibers(before/after) to the original one, take mean distance and std as measurement

## downsample the fibers and get tracks_before, tracks_after, tracks_original

In [38]:
from dipy.tracking.metrics import downsample
import random
tracks_after = [downsample(fib_after[i][0],7) for i in range(len(fib_after))]
tracks_original = [downsample(fib_original[i][0],7) for i in range(len(fib_original))]
## generate a random list to get equal number of fibers from fib before COMMIT
list = range(len(fib_before))
slice1 = random.sample(list, len(fib_after))
tracks_before1 = [downsample(fib_before[i][0],7) for i in slice1]

In [39]:
from dipy.segment.bundles import bundles_distances_mdf
DM_before1 = bundles_distances_mdf(tracks_before1, tracks_original)
minD_before1 = [min(DM_before1[i,:]) for i in range(len(DM_before1))]
DM_after = bundles_distances_mdf(tracks_after, tracks_original)
minD_after = [min(DM_after[i,:]) for i in range(len(DM_after))]

## compare tracks_after and tracks_before with tracks_original with mean distance and STD

In [40]:
meanD_before= np.mean(minD_before1)
stdD_before= np.std(minD_before1)

meanD_after= np.mean(minD_after)
stdD_after= np.std(minD_after)

In [41]:
meanD_after

0.19444568598397008

In [42]:
meanD_before

0.1945705805421867

## Compare on a voxel level

original resolution -- 0.015mm

In [43]:
for items in range(len(fib_before)-1,-1,-1):
    if items not in slice1:
        del fib_before[items]
    j=j+1
nib.trackvis.write('fibers_before.trk',fib_before,hdr_before)

In [44]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk')

In [45]:
CMD = 'TractConverter.py -i fibers_before.trk -o fibers_before.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [46]:
CMD='tckmap fibers_before.tck syn_CLARITY_random.nii.gz -vox 0.015' 
subprocess.call(CMD, shell = True)

0

In [47]:
data_original = nib.load('syn_CLARITY.nii.gz')
data_before = nib.load('syn_CLARITY_random.nii.gz')
data_after = nib.load('syn_CLARITY_after.nii.gz')
original = data_original.get_fdata()
random = data_before.get_fdata()
after = data_after.get_fdata()

## normalization
sum_original = sum(sum(sum(data_original.get_fdata())))
sum_before = sum(sum(sum(data_before.get_fdata())))
sum_after = sum(sum(sum(data_after.get_fdata())))
volume_original = data_original.get_fdata()/sum_original
volume_before = data_before.get_fdata()/sum_before
volume_after = data_after.get_fdata()/sum_after

## correlation measure to conduct grey level matching

In [71]:
sum_original = sum(sum(sum(data_original.get_fdata())))
sum_before = sum(sum(sum(data_before.get_fdata())))
sum_after = sum(sum(sum(data_after.get_fdata())))

In [72]:
center_original = (original-sum_original/(191*178*39)).flatten()
center_before = (random - sum_before/(191*178*39)).flatten()
center_after = (after - sum_after/(191*178*39)).flatten()

In [73]:
module_original = np.sqrt(np.dot(center_original,center_original))
module_before = np.sqrt(np.dot(center_before,center_before))
module_after = np.sqrt(np.dot(center_after,center_after))

In [74]:
random_coef = np.dot(center_original.flatten(),center_before.flatten())/(module_original*module_before)
COMMIT_coef = np.dot(center_original.flatten(),center_after.flatten())/(module_original*module_after)

In [75]:
print(random_coef)
print(COMMIT_coef)

0.07088778655363923
0.07072043871692317


In [ ]:
square_error_before = compClarity(volume_before, volume_original)
square_error_after = compClarity(volume_after, volume_original)

## change the resolution here

resolution based on mean distance after COMMIT

CMD='tckmap origin0.015.tck syn_CLARITY_COMMIT_res.nii.gz -vox 0.195' 
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_after.tck syn_CLARITY_after_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_before.tck syn_CLARITY_random_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

data_original_COMMIT_res = nib.load('syn_CLARITY_COMMIT_res.nii.gz')
data_before_COMMIT_res = nib.load('syn_CLARITY_random_COMMIT_res.nii.gz')
data_after_COMMIT_res = nib.load('syn_CLARITY_after_COMMIT_res.nii.gz')
## normalization
sum_original_COMMIT_res = sum(sum(sum(data_original_COMMIT_res.get_fdata())))
sum_before_COMMIT_res = sum(sum(sum(data_before_COMMIT_res.get_fdata())))
sum_after_COMMIT_res = sum(sum(sum(data_after_COMMIT_res.get_fdata())))
volume_original_COMMIT_res = data_original_COMMIT_res.get_fdata()/sum_original_COMMIT_res
volume_before_COMMIT_res = data_before_COMMIT_res.get_fdata()/sum_before_COMMIT_res
volume_after_COMMIT_res = data_after_COMMIT_res.get_fdata()/sum_after_COMMIT_res

square_error_before_COMMIT_res = compClarity(volume_before_COMMIT_res, volume_original_COMMIT_res)
square_error_after_COMMIT_res = compClarity(volume_after_COMMIT_res, volume_original_COMMIT_res)

## Compare on a voxel level - binary

original resolution -- 0.015mm

In [ ]:
def binarize(volume):
    volume_prime = np.zeros([len(volume),len(volume[1]),len(volume[1][1])])
    for x in range(len(volume)):
        for y in range(len(volume[1])):
            for z in range(len(volume[1][1])):
                if volume[x,y,z]!=0:
                    volume_prime[x,y,z]=1
    return volume_prime

In [ ]:
Volume_Original = binarize(volume_original)
Volume_Before = binarize(volume_before)
Volume_After = binarize(volume_after)
error_before = compClarity(Volume_Before, Volume_Original)
error_after = compClarity(Volume_After, Volume_Original)


## change the resolution here

COMMIT resolution 0.186

Volume_Original_COMMIT_res = binarize(volume_original_COMMIT_res)
Volume_Before_COMMIT_res = binarize(volume_before_COMMIT_res)
Volume_After_COMMIT_res = binarize(volume_after_COMMIT_res)
error_before_COMMIT_res = compClarity(Volume_Before_COMMIT_res, Volume_Original_COMMIT_res)
error_after_COMMIT_res = compClarity(Volume_After_COMMIT_res, Volume_Original_COMMIT_res)

## save results

In [ ]:
cd ..

In [ ]:
f = open('87.txt','a')  
f.write('group: Nboffibers_before: ' + str(number))
f.write('         Nboffibers_after: ' + str(len(fib_after)))
f.write('\n\n')
f.close()

f = open('87_TRACT_random_mean.txt','a')  
f.write(str(meanD_before)+', ')
f.close()

f = open('87_TRACT_random_std.txt','a') 
f.write(str(stdD_before)+', ')
f.close()


f = open('87_TRACT_COMMIT_mean.txt','a')
f.write(str(meanD_after)+', ')
f.close()

f = open('87_TRACT_COMMIT_std.txt','a')
f.write(str(stdD_after)+', ')
f.close()

In [ ]:
f = open('87_VOXEL_B_random.txt','a')
f.write(str(error_before)+', ')
f.close()

f = open('87_VOXEL_B_COMMIT.txt','a')
f.write(str(error_after)+', ')
f.close()

In [ ]:
f = open('87_VOXEL_NB_random.txt','a')
f.write(str(square_error_before)+', ')
f.close()

f = open('87_VOXEL_NB_COMMIT.txt','a')
f.write(str(square_error_after)+', ')
f.close()

In [ ]:
f = open('87_VOXEL_NB_random.txt','a')
f.write(str(random_coef)+', ')
f.close()

f = open('87_VOXEL_NB_COMMIT.txt','a')
f.write(str(COMMIT_coef)+', ')
f.close()

f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res - BINARIZATION:')
f.write('\nerror random: ' + str(error_before_COMMIT_res))
f.write('\nerror COMMIT: ' + str(error_after_COMMIT_res)) 
f.write('\n\n')
f.close()

f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res- non-BINARIZATION:')
f.write('\nsquare error random: ' + str(square_error_before_COMMIT_res))
f.write('\nsquare error COMMIT: ' + str(square_error_after_COMMIT_res)) 
f.write('\n\n\n\n\n')
f.close()

In [ ]:
os.rename('results_script1','%d'%order)